# Utils

In [24]:
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt


imgCol = cv2.imread('../Utils/LenaJPEG.jpg')
imgCol.shape

def plotSignal(signal=[], title='', xmin = 0, xmax = 0):
  if(xmax == 0):
    xmax = signal.size

  plt.title(title)
  plt.plot(signal)
  plt.xlim(xmin, xmax)
  plt.show() 

def show_Image(img=[], resize=-1, title='img'):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    if 0 < resize < 2:
      img = cv2.resize(img, [int(width*resize), int(height*resize)])
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def get_Image(path='../Utils/LenaJPEG.jpg', resize=-1):
    toRet = cv2.imread(path)
    if 0 < resize < 2:
      if len(toRet.shape) == 3:
          height, width, depth = toRet.shape
      else :
          height, width = toRet.shape
      toRet = cv2.resize(toRet, [int(width*resize), int(height*resize)])
    return toRet
   

# Praktikum 01 (Not necessary)

In [4]:
def printNumberFrom1To100():
    for i in range(1, 101):
        print(i)

def fibunacci(n):
    a = [0]*(n+4)
    a[0] = 1
    a[1] = 1
    print(0)
    print(1)
    print(1)
    for i in range(2,n+3):
        a[i] = a[i-1]+a[i-2]
        print(a[i])

# fibunacci(10)

# for y in range(height):
#     for x in range(width):
#         imgCol[y][x] = pow(imgCol[y][x]/255, 1/2) * 255



# cv2.imshow("img", imgCol)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


# Praktikum 02

In [3]:
def getHistogramm(img=[]):
    histogram=np.zeros(256)
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    for y in range(height):
        for x in range(width):
            if img.ndim == 3:
                i = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            else:
                i = img[y][x]

            histogram[int(i)] += 1

    return histogram

def getNormHistogramm(img=[], h=[]):
    if len(h)==0:
        if len(img.shape) == 3:
            height, width, depth = img.shape
        else :
            height, width = img.shape
        return getHistogramm(img) / (width*height)
    else:
        return h/np.sum(h)

def getCumulatedHistogramm(img=[], h=[]):
    H = np.zeros(256)
    if len(h)==0:
        h = getNormHistogramm(img)
    else:
        h = getNormHistogramm(h=h)
    for i in range(len(h)):
        for j in range(i): 
            H[i] += h[j]
    return H

def linearContrast(img=[], t0=0, t1=255):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    toRet = np.zeros((height, width), dtype='uint8')
    for y in range(height):
        for x in range(width):
            if img.ndim == 3:
                a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            else:
                a = img[y][x]
            if a < t0:
                toRet[y][x] = 0
            if t0 <= a < t1:
                toRet[y][x] = 255/(t1-t0) * (a-t0)
            if a >= t1:
                toRet[y][x] = 255
    return toRet

def autoContrast(img, schwellwert):
    a1_low = getPercentage(img, schwellwert)
    a1_high = getPercentage(img, 1-schwellwert)
    return linearContrast(img, a1_low, a1_high)

def getPercentage(img, schwellwert):
    if schwellwert < 0:
        schwellwert = 0
    elif schwellwert > 1:
        schwellwert = 1
    H = getCumulatedHistogramm(img)
    toRet = 255
    for i in range(len(H)):
        if H[i] >= schwellwert:
            toRet = i
            break
    return toRet

def getMedianOfHistogramm(img=[], h=[]):
    if len(h) == 0:
        H = getCumulatedHistogramm(img)
    else:
        H = getNormHistogramm(h=h)
    for i in range(len(H)):
        if H[i] >= 0.5:
            return i


def getBinary(img=[], schwellwert=125):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    toRet = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            if a < schwellwert:
                toRet[y][x] = 0
            else:
                toRet[y][x] = 1
    return toRet

def same_image(img=[]):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    toRet = np.zeros((width, height), dtype='uint8')
    for y in range(height):
        for x in range(width):
            a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            # a = img[y][x][0]
            toRet[y][x] = a
            
    return toRet

testing Praktikum 02

In [6]:
# print(getMedianOfHistogramm(imgCol))
median = getMedianOfHistogramm(imgCol)
print(median)
img = getBinary(imgCol, median)
# getNormHistogramm(img)
# auto_con = autoContrast(imgCol, 0.1)

# plotSignal(getCumulatedHistogramm(imgCol))
show_Image(img)
# show_Image(imgCol)

128


# Praktikum 03

In [4]:
# Not working
def Otsu_Threshold_naiv(Histrogramm=[], img=[]):
    toRet = -1
    varMax = -1
    width, height, depth = img.shape
    size = width * height
    K = len(Histrogramm)
    for i in range(K-1):
        w_0 = np.sum(Histrogramm[0:i]) / size
        w_1 = np.sum(Histrogramm[i:K]) / size
        my_0 = np.mean(Histrogramm[0:i])
        my_1 = np.mean(Histrogramm[i:K])
        varBetween = w_0 * w_1 * (my_0-my_1)**2
        if varMax < varBetween:
            varMax = varBetween
            toRet = i
    return toRet

def weigthed_Histogram(h=[]):
    toRet = 0
    for i in range(len(h)):
        toRet += i * h[i]
    return toRet


def Otsu_Threshold_efficient(img=[], vorwissen=False, t=1):
    """
    calculate the treshold for an linear regression

    :param img: the used Image
    :type img: Arraylike
    :param vorwissen: if you know how many persentage of the picture will be black
    :type vorwissen: boolean
    :param t: the known persentage, if vorwissen is True
    :type t: float in range [0;1]
    :return: treshold for further computing
    :rtype: int
    """
    h = getHistogramm(img)
    toRet = -1
    varMax = -1
    c_0 = 0
    sum_0 = 0 
    width, height, depth = img.shape
    size = width * height
    if vorwissen:
        K = getPercentage(img, t)
    else:
        K = len(h)
    for i in range(K):
        c_0 = c_0 + h[i]
        c_1 = size - c_0
        sum_0 = sum_0 + i * h[i]
        my_0 = sum_0 / c_0
        my_1 = ( weigthed_Histogram(h) - sum_0) / c_1
        varBetween = c_0 * c_1 * (my_0-my_1)**2
        if varMax < varBetween:
            varMax = varBetween
            toRet = i
    return toRet

def getEroded(img=[], schwellwert=125):
    toRet = getBinary(img, schwellwert)
    kernel = np.array([[0, 1, 0],
                       [1, 1, 1],
                       [0, 1, 0]],
                      dtype=np.uint8)
        
    toRet = cv2.erode(toRet, kernel)
    toRet = cv2.erode(toRet, kernel)
    toRet = cv2.dilate(toRet, kernel)
    return toRet

def region_labeling(img=[]):
    toRet = np.array(img, dtype=np.uint8)
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    m = 2
    for y in range(height):
        for x in range(width):
            if img[y][x] == 1:
                toRet = floodfill(img, x, y, m)
                m+=1
    for em in range(m):
        print('region_label: m:',em,'count:',np.count_nonzero(toRet == em))
    return toRet

def floodfill(img=[], x=0, y=0, m=1):
    toRet = img
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    if(x == 0 and y == 0):
        print(height, width)
    S = []
    S.append((x, y))
    
    while len(S) > 0:
        (x, y) = S.pop()
        if((0<=x<width and 0<=y<height) and img[y][x] == 1):
            toRet[y][x] = m
            S.append((x+1, y))
            S.append((x, y+1))
            S.append((x, y-1))
            S.append((x-1, y))
    return toRet

def computeProperties(img):
    R = {}
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    for y in range(height):
        for x in range(width):
            m = img[y][x]
            if(m >= 2 and m not in R):
                R[m] = {'flaeche': 0, 'bb': {'xmin': x, 'xmax': x, 'ymin': y, 'ymax': y}}
                R[m]['flaeche'] += 1
                R[m]['bb']['xmin'] = min(R[m]['bb']['xmin'], x)
                R[m]['bb']['xmax'] = max(R[m]['bb']['xmax'], x)
                R[m]['bb']['ymin'] = min(R[m]['bb']['ymin'], y)
                R[m]['bb']['ymax'] = max(R[m]['bb']['ymax'], y)

                R[m]['bb']['height'] = R[m]['bb']['ymax'] - R[m]['bb']['ymin']
                R[m]['bb']['width'] = R[m]['bb']['xmax'] - R[m]['bb']['xmax']

    return R

def test_Properties(img=[]):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    for y in range(height):
        for x in range(width):
            m = img[y][x]
            if(m>=2):
                img[y][x] = 130
    return img




testing Praktikum 03

In [25]:
# SetExmpl = cv2.imread('../Utils/SetWellen.png')
SetExmpl = get_Image('../Utils/SetWellen.png', resize=0.5)

otsu_efficient = Otsu_Threshold_efficient(SetExmpl)
print(otsu_efficient)

115


C:\Users\Johan Laptop\AppData\Local\Temp\ipykernel_8400\1215159681.py:54: RuntimeWarning: invalid value encountered in scalar divide
  my_0 = sum_0 / c_0
C:\Users\Johan Laptop\AppData\Local\Temp\ipykernel_8400\1215159681.py:55: RuntimeWarning: invalid value encountered in scalar divide
  my_1 = ( weigthed_Histogram(h) - sum_0) / c_1


In [26]:

binary = getEroded(SetExmpl, 130)
label = region_labeling(binary)
gray_image = linearContrast(label, 0, 255)
tst = test_Properties(gray_image)
# R = computeProperties(gray_image)
show_Image(tst)

region_label: m: 0 count: 252289
region_label: m: 1 count: 0
region_label: m: 2 count: 17825
region_label: m: 3 count: 20028
region_label: m: 4 count: 16274
region_label: m: 5 count: 15850
region_label: m: 6 count: 17704
region_label: m: 7 count: 17968


In [129]:
test_img = np.ones((255,255), dtype=np.uint8)
for y in range(255):
    for x in range(255):
        if(abs(x-y)<2):
            test_img[y][x] = 255
test_img = region_labeling(test_img)
show_Image(test_img)